In [1]:
import numpy as np
import torch
import scipy
import time
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import nbimporter
import unittest

import torch.nn.functional as f

from utils import *


In [2]:
image_pil = Image.open('main.jpg').convert('L')    # converting rgb image to grayscale image
image = np.array(image_pil)                       # converted image into numpy array

KERNEL = np.array(
    [
        [1, 0, -1], 
        [2, 0, -2],
        [1, 0, -1]
    ])


In [3]:
def filter_scipy_convolve2d(img, kernel):
    start =time.time()
    filtered_image = np.zeros_like(img).astype(np.float64)   # filtered image variable array of size same as input image
    
    #casted into double data type using float 64
    
    filtered_image = scipy.signal.convolve2d(img, kernel, mode='same')

    print("Elapsed time (s)=", time.time() - start)
    return filtered_image

In [4]:
img = filter_scipy_convolve2d(image, KERNEL)
writeDoubleImage(img, "scipy.jpg")

Elapsed time (s)= 7.865091800689697


In [23]:
# part 1.2
def filter_numpy_for_loop(img, kernel):
    start = time.time()
    filtered_image = np.zeros_like(img).astype(np.float64)
    # flip my kernel for better alignment with image
    

    # image and kernel dimesnions to run for loop iteration for convolution
   

    # estimating padding size 
    # pad_h = k_h // 2
    # pad_w = k_w // 2

    #padded_image = np.pad(img,pad_width=pad_size,mode='constant',constant_values=0)
    padded_image = np.pad(img, pad_width =1, mode='constant', constant_values=0)
    K_flip = np.flip(kernel)
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):

            # what part of image will be convoluted
            # convolute = padded_image[i:i+k_h,j:j+k_w]
            # apply convolution - step 1 - element wise multiplication of kernel and convolute area ; step 2: summation
            #prod = convolute * K_flip  # step 1

            filtered_image[i,j] = np.sum(padded_image[i:i+3,j:j+3] * K_flip)  # step 2

    print("Elapsed time (s)=", time.time() - start)
    return filtered_image

# Part 1.2
# def filter_numpy_for_loop(img, kernel):
#     start = time.time()
#     filtered_image = np.zeros_like(img).astype(np.float64)
#     # flip my kernel for better alignment with image
    

#     # image and kernel dimesnions to run for loop iteration for convolution
   

    # estimating padding size 
    # pad_h = k_h // 2
    # pad_w = k_w // 2

    #padded_image = np.pad(img,pad_width=pad_size,mode='constant',constant_values=0)
    # padded_image = np.pad(img, pad_width =1, mode='constant', constant_values=0)
    # K_flip = np.flip(kernel)
    
    # for i in range(img.shape[0]):
    #     for j in range(img.shape[1]):

    #         # what part of image will be convoluted
    #         # convolute = padded_image[i:i+k_h,j:j+k_w]
    #         # apply convolution - step 1 - element wise multiplication of kernel and convolute area ; step 2: summation
    #         #prod = convolute * K_flip  # step 1

    #         filtered_image[i,j] = np.sum(padded_image[i:i+3,j:j+3] * K_flip)  # step 2

    # print("Elapsed time (s)=", time.time() - start)
    # return filtered_image



In [24]:
img1 = filter_numpy_for_loop(image, KERNEL)
writeDoubleImage(img1, "numpy_for_loop.jpg")

Elapsed time (s)= 19.600974321365356


In [65]:
# part 1.3

def filter_torch(img, kernel):
    start = time.time()
    flip_kernel = np.flip(kernel).copy()
    conv_layer = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1) # One input channel (grayscaled img), one output channel (one kernel), kernel_size =3
    weight_tensor = torch.tensor(flip_kernel, dtype=torch.float64)

    with torch.no_grad():  # To Hardcode values, this ensures the weights dont update due to gradient tracking
      conv_layer.weight.data = weight_tensor.unsqueeze(0).unsqueeze(0) #Incrase dim of tensor to indicate batch size,
      conv_layer.bias = torch.nn.Parameter(torch.tensor([0.0], dtype=torch.float64)) #Ensuring bias has the same dimensions as weights
    
    padded_img = np.pad(img, pad_width=1, mode='constant', constant_values=0)
    img_tensor = torch.tensor(padded_img, dtype=torch.float64) # Convert img to tensor with dtype = double
    img_input = img_tensor.unsqueeze(0).unsqueeze(0) # Increasing the dimension of the tensor to indicate batch size to pytorch
    #filtered_image = conv_layer(torch.tensor(img_input, dtype=torch.float64))
    filtered_image = conv_layer(img_input.double()).squeeze()

    print("Elapsed time (s)=", time.time() - start)
    return filtered_image.detach().numpy()


# def filter_torch(img, kernel):
#     start = time.time()

#     K_flip = np.flip(kernel)  # flipped numpy kernel

#     # unsqueeze is used to add dimension to tensor at a given position
#     # for conv2d function, we need a dimension for channels and batch dimnsion -- hence repeated twice
#     # converted to float to ensure correct data type and easy operation
    
#     tensor_image = torch.from_numpy(img).float().unsqueeze(0).unsqueeze(0)  # converting image numpy array to tensor
#     tensor_kernel = torch.from_numpy(K_flip.copy()).float().unsqueeze(0).unsqueeze(0)  # tensor kernel 
    
# #     tensor_kernel = tensor_kernel.unsqueeze(0)   # input format for conv2d function

# #     tensor_image = tensor_image.unsqueeze(0)
    
#     #print(tensor_image.size())

#     filtered_image = np.zeros_like(img).astype(np.float64)
#     filtered_image = f.conv2d(tensor_image, tensor_kernel, padding =1).squeeze(0).squeeze(0)  # removing added dimension ensuring output image is of same size as input image
    
    
#     print("Elapsed time (s)=", time.time() - start)
   
#     return filtered_image.detach().numpy()   # tensor to numpy array

In [66]:
img2 = filter_torch(image, KERNEL)
writeDoubleImage(img2, "torch_conv.jpg")

Elapsed time (s)= 0.029062986373901367


[[-1  0  1]
 [-2  0  2]
 [-1  0  1]]
